In [1]:
__name__ = "k1lib.callbacks"

In [2]:
#export
from .callbacks import Callback, Callbacks, Cbs
import k1lib, time
__all__ = ["BatchLimit", "EpochLimit", "TimeLimit", "CancelOnExplosion",
           "CancelOnLowLoss", "CancelOnHighAccuracy", "DontTrain",
           "GradientClipping", "GradientClippingNorm", "TrainOnly", "ValidOnly"]

In [3]:
#export
@k1lib.patch(Cbs)
class BatchLimit(Callback):
    """Cancels the epoch after executed certain number of batches"""
    def __init__(self, limit:int):
        super().__init__(); self.order = 25
        self.limit = limit if limit != None else float("inf")
    def startEpoch(self): self.currentBatch = 0
    def startBatch(self):
        if self.currentBatch >= self.limit:
            raise k1lib.CancelEpochException(f"Batch {self.limit} reached")
    def endBatch(self): self.currentBatch += 1

In [4]:
#export
@k1lib.patch(Cbs)
class EpochLimit(Callback):
    """Cancels the run after executed certain number of epochs"""
    def __init__(self, limit:int):
        super().__init__(); self.order = 25
        self.limit = limit if limit != None else float("inf")
    def startRun(self): self.currentEpoch = 0
    def startEpoch(self):
        if self.currentEpoch >= self.limit:
            raise k1lib.CancelRunException(f"Epoch {self.limit} reached!")
    def endEpoch(self): self.currentEpoch += 1

In [5]:
#export
@k1lib.patch(Cbs)
class TimeLimit(Callback):
    """Cancels the run after a certain number of seconds have passed"""
    def __init__(self, seconds=30):
        super().__init__(); self.seconds = seconds; self.order = 25
    def startRun(self): self.startTime = time.time()
    def startBatch(self):
        if time.time() - self.startTime > self.seconds:
            raise k1lib.CancelRunException(f"Takes more than {self.seconds} seconds!")

In [6]:
#export
@k1lib.patch(Cbs)
class CancelOnExplosion(Callback):
    """Cancels the run if any of the parameters are larger than a certain limit"""
    def __init__(self, limit:float=1e6):
        super().__init__(); self.order = 25
        self.limit = limit; self.triggered = False
    def startRun(self): self.triggered = False
    def startBatch(self):
        for p in self.l.model.parameters():
            o = p.detach()
            if o.max().float() > self.limit or o.min().float() < -self.limit:
                self.triggered = True
                raise k1lib.CancelRunException("Explosion detected!")
    def __repr__(self):
        return f"""{self._reprHead}, use...
- cb.triggered: to see if there was an explosion on the last run
- cb.progress: to see current progress at explosion time
{self._reprCan}"""

In [7]:
#export
@k1lib.patch(Cbs)
class CancelOnLowLoss(Callback):
    " "
    def __init__(self, loss:float, epochMode:bool=False):
        """Cancels the run if loss is lower than amount specified.
Original class: :class:`~k1lib.callbacks.limits.CancelOnLowLoss`

:param epochMode: False if use batch loss, True if use valid epoch loss"""
        super().__init__(); self.order = 25; self.dependsOn = ["Loss"]
        self.loss = loss; self.epochMode = epochMode
    def startRun(self):
        if not hasattr(self.l.cbs, "Loss"):
            raise AttributeError("Learner does not have required `Loss` callback")
        self.v = self.cbs.Loss.valid; self.ve = self.cbs.Loss.epoch.valid # List[int]
    def endBatch(self):
        if self.epochMode:
            if len(self.ve) > 0 and self.ve[-1] < self.loss:
                raise k1lib.CancelRunException(f"Low loss {self.loss} ({self.ve[-3:]} actual) achieved!")
        elif len(self.v) and self.v[-1] < self.loss:
            raise k1lib.CancelRunException(f"Low loss {self.loss} ({self.v[-3:]} actual) achieved!")

In [8]:
#export
@k1lib.patch(Cbs)
class CancelOnHighAccuracy(Callback):
    """Cancels the run if accuracy is higher than the amount specified"""
    def __init__(self, accuracy:float):
        super().__init__(); self.order = 25
        self.accuracy = accuracy; self.dependsOn = ["Accuracy"]
    def endBatch(self):
        if not hasattr(self.l, "Accuracy"): raise AttributeError("Learner does not have `Accuracy` callback")
        a = self.l.Accuracy.valid[-1]
        if a > self.accuracy:
            raise k1lib.CancelRunException(f"High accuracy {self.accuracy} ({a} actual) achieved!")

In [9]:
#export
@k1lib.patch(Cbs)
class DontTrain(Callback):
    """Don't allow the network to train at all"""
    def startBackward(self): return True
    def startStep(self): return True

In [10]:
#export
from torch.nn.utils import clip_grad_value_
@k1lib.patch(Cbs)
class GradientClipping(Callback):
    """Clips gradient to a specific max value"""
    def __init__(self, value:float): super().__init__(); self.value = value
    def startStep(self):
        clip_grad_value_(self.l.model.parameters(), self.value)

In [11]:
#export
from torch.nn.utils import clip_grad_norm_
@k1lib.patch(Cbs)
class GradientClippingNorm(Callback):
    """Clips gradient to a specific max_norm value. Can choose to lump
all params together or do each separately.

See also: :class:`~k1lib.callbacks.limits.GradientClipping` callback."""
    def __init__(self, max_norm:float, each:bool=True):
        super().__init__(); self.max_norm = max_norm; self.each = each
    def startStep(self):
        if self.each:
            for m in self.l.model.parameters():
                clip_grad_norm_(m, self.max_norm)
        else: clip_grad_norm_(self.l.model.parameters(), self.max_norm)

In [12]:
#export
@k1lib.patch(Cbs)
class TrainOnly(Callback):
    " "
    def __init__(self, cb):
        """Only executes specified callback when training. This modifies the callback's
``suspended`` variable, so it may interfere with :meth:`k1lib.callbacks.callbacks.Callbacks.suspend`
by setting it to different values while in the context."""
        super().__init__(); self.cb = cb
    def startBatch(self):
        self.cb.suspended = not self.l.model.training
@k1lib.patch(Cbs)
class ValidOnly(Callback):
    " "
    def __init__(self, cb):
        """Same as :class:`TrainOnly`, but only executes specified callback when doing
validation."""
        super().__init__(); self.cb = cb
    def startBatch(self):
        self.cb.suspended = self.l.model.training

In [14]:
!../../export.py callbacks/limits

Current dir: /home/kelvin/repos/labs/k1lib, ../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.11
Uninstalling k1lib-0.11:
  Successfully uninstalled k1lib-0.11
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/fmt.py -> build/lib/k1lib
copying k1lib/_context.py -> build/lib/k1lib
copying k1lib/selector.py -> build